In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install PyYAML==5.1

In [ ]:
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

In [ ]:
# %cd '/content/drive/MyDrive/UpWork/Protein/'
# !git clone https://github.com/google/automl.git
# !mv automl/efficientdet .
# !rm -r automl
# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
# !tar zxf efficientdet-d0.tar.gz
# %cd /content

In [ ]:
# Split test images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/test_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations \
  -o /content/drive/MyDrive/UpWork/Protein/Data/test_split_images \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -d 768 -p 8

In [ ]:
# Split validation images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/validation_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations \
  -o /content/drive/MyDrive/UpWork/Protein/Data/validation_split_images \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -d 768 -p 8

In [ ]:
# Split train images
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/data_preparation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/train_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations \
  -o /content/drive/MyDrive/UpWork/Protein/Data/train_split_images \
  -l /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -d 768 -p 8

In [ ]:
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/train_split_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -o /content/drive/MyDrive/UpWork/Protein/Data/train_split_annotation.json
  
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/validation_split_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -o /content/drive/MyDrive/UpWork/Protein/Data/validation_split_annotation.json

!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/src:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/src/coco_annotation.py \
  -i /content/drive/MyDrive/UpWork/Protein/Data/test_split_images/ \
  -a /content/drive/MyDrive/UpWork/Protein/Data/annotations_split \
  -o /content/drive/MyDrive/UpWork/Protein/Data/test_split_annotation.json

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/train_split_images \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/train_split_annotation.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_train_split \
  --num_shards=6

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/validation_split_images \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/validation_split_annotation.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split \
  --num_shards=2

In [ ]:
# Create TF Records
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/dataset/create_coco_tfrecord.py --logtostderr \
  --image_dir=/content/drive/MyDrive/UpWork/Protein/Data/test_split_images \
  --object_annotations_file=/content/drive/MyDrive/UpWork/Protein/Data/test_split_annotation.json \
  --output_file_prefix=/content/drive/MyDrive/UpWork/Protein/tf_data1/protein_test_split \
  --num_shards=2

In [ ]:
# Training
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py --mode=train_and_eval \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--train_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_train_split-*-of-00006.tfrecord \
	--val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split-*-of-00002.tfrecord \
	--model_name=efficientdet-d0 \
	--model_dir=/content/drive/MyDrive/UpWork/Protein/model_2/ \
	--ckpt=/content/drive/MyDrive/UpWork/Protein/model_1 \
	--train_batch_size=4 \
	--num_examples_per_epoch=3388  --num_epochs=10  \
  --eval_batch_size=4 --eval_samples=1220  \

2021-08-24 11:19:10.148030: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
I0824 11:19:12.848766 140298132957056 main.py:228] {'name': 'efficientdet-d0', 'act_type': 'swish', 'image_size': (512, 512), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': 'v0', 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 1, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': {1: 'Protein'}, 'max_instances_per_image': 150, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epoch

In [ ]:
# Evalute on validation data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
  --mode=eval \
  --model_name=efficientdet-d0 \
  --model_dir=/content/drive/MyDrive/UpWork/Protein/model_2/	\
  --val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_validation_split-*-of-00002.tfrecord \
  --eval_batch_size=4  --eval_samples=1220

In [ ]:
# Evalute on test data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/main.py \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
  --mode=eval \
  --model_name=efficientdet-d0 \
  --model_dir=/content/drive/MyDrive/UpWork/Protein/model_2/	\
  --val_file_pattern=/content/drive/MyDrive/UpWork/Protein/tf_data/protein_test_split-*-of-00002.tfrecord \
  --eval_batch_size=4 --eval_samples=995

2021-08-24 13:49:09.655594: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
I0824 13:49:12.446262 139929035200384 main.py:228] {'name': 'efficientdet-d0', 'act_type': 'swish', 'image_size': (512, 512), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': 'v0', 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 1, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': {1: 'Protein'}, 'max_instances_per_image': 150, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epoch

In [ ]:
# Exporting
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py --runmode=saved_model \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --ckpt_path=/content/drive/MyDrive/UpWork/Protein/model_2/model.ckpt-3388 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_2-4/	

2021-08-24 14:00:59.325393: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-24 14:01:03.888204: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-24 14:01:03.889396: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-24 14:01:03.900325: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-24 14:01:03.900373: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c6dac7f2014b): /proc/driver/nvidia/version does not exist
W0824 

In [ ]:
# Inference on test data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py \
  --runmode=saved_model_infer \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_2-4/	\
  --input_image=/content/drive/MyDrive/UpWork/Protein/Data/test_split_images/TcdA1-*_frames*.png \
  --output_image_dir=/content/drive/MyDrive/UpWork/Protein/output_test_model2_e4/ \
	--min_score_thresh=0.5 \
	--max_boxes_to_draw=150

2021-08-24 14:05:03.297418: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-24 14:05:06.213535: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-24 14:05:06.214712: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-24 14:05:06.225550: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-24 14:05:06.225593: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c6dac7f2014b): /proc/driver/nvidia/version does not exist
Instru

In [ ]:
# Inference on training data
!PYTHONPATH="/content/drive/MyDrive/UpWork/Protein/efficientdet:$PYTHONPATH" \
  python /content/drive/MyDrive/UpWork/Protein/efficientdet/model_inspect.py \
  --runmode=saved_model_infer \
  --hparams=/content/drive/MyDrive/UpWork/Protein/myconfig.yaml \
	--model_name=efficientdet-d0 \
  --saved_model_dir=/content/drive/MyDrive/UpWork/Protein/exported_model_1/	\
  --input_image=/content/drive/MyDrive/UpWork/Protein/train_images/TcdA1-0011_frames.png \
  --output_image_dir=output_train/ \
	--min_score_thresh=0.5 \
	--max_boxes_to_draw=50